In [1]:
import numpy as np
import cvxopt
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches
from matplotlib.patches import Polygon
import matplotlib.cm as cmx
import matplotlib.colors as colors

In [5]:
del_t = 0.1
v_init = 0.1
w_init = 0

class agent:
    def __init__(self,rad,start,goal,theta,vmax,wmax):
        self.rad = rad
        self.time = 0
        self.goal = goal
        self.theta = theta
        self.v = v_init
        self.w = w_init
        self.pos = start
        self.vmax = vmax
        self.wmax = wmax
        self.optfl = 0
    def __str__(self):
        return (str(self.time)+" "+str(self.theta)+" "+str(self.v)+" "+str(self.w)+" "+str(self.pos))
    def opt_traj(self):
        self.visualize_traj()
        cost_hist = []
        cost_hist.append(0)
        it = 0
        while (np.linalg.norm(self.pos-self.goal)>0.5):
#             self.visualize_traj()
            if (self.optfl==0):
                theta_new = self.theta + self.w*del_t
                xgi = self.pos[0] + self.v*del_t*np.cos(theta_new) - self.goal[0]
                ygi = self.pos[1] + self.v*del_t*np.sin(theta_new) - self.goal[1]
                p_mat = 2*np.array([[del_t**2,0],
                                  [0,(self.v**2)*(del_t**4)]])
                P = cvxopt.matrix(p_mat,tc='d')
                q_mat = np.array([2*del_t*(xgi*np.cos(theta_new)+ygi*np.sin(theta_new)),
                                 -2*del_t**2*self.v*(xgi*np.sin(theta_new)-ygi*np.cos(theta_new))])
    #             print (p_mat,q_mat)
                Q = cvxopt.matrix(q_mat,tc='d')
                g_mat = np.array([[1,0],[0,1],[0,-1]])
                h = cvxopt.matrix(np.array([self.vmax-self.v,self.wmax-self.w,self.wmax-self.w]),tc='d')
                g = cvxopt.matrix(g_mat,tc='d')
                sol = cvxopt.solvers.qp(P,Q,g,h,options={'show_progress': False})
                v_new = sol['x'][0]
                w_new = sol['x'][1]
                print ("v_new: ",v_new,"w_new: ",w_new)
                it+=1
                x_new = self.pos[0] + (self.v+v_new)*del_t*np.cos(self.theta+(self.w+w_new)*del_t)
                y_new = self.pos[1] + (self.v+v_new)*del_t*np.sin(self.theta+(self.w+w_new)*del_t)
                cost = np.linalg.norm(np.array([x_new,y_new])- self.goal)**2
                prev_cost = cost_hist[-1]
                print (it,cost)
                if (np.linalg.norm(np.array([v_new,w_new]))>0.1):
                    self.v += v_new 
                    self.w += w_new
                    print (self.v,self.w)
                    cost_hist.append(cost)
                else:
                    self.optfl = 1
                    self.v += v_new 
                    self.w += w_new
            else:
                x_new = self.pos[0] + self.v*del_t*np.cos(self.theta+self.w*del_t)
                y_new = self.pos[1] + self.v*del_t*np.sin(self.theta+self.w*del_t)
                self.pos = [x_new,y_new]
                print (self)
                self.theta = self.theta+w_new*del_t
                self.time+=1
                self.visualize_traj()
                
             
    def visualize_traj(self):
        figure = plt.figure()
#         print (self.pos)
        ax = figure.add_subplot(1,1,1)
        robot = matplotlib.patches.Rectangle(
            (self.pos[0]-self.rad*np.sqrt(2)*np.cos(self.theta+np.pi/4),self.pos[1]-self.rad*np.sqrt(2)*np.sin(self.theta+np.pi/4)),
            height = self.rad*2,
            width = self.rad*2,
            angle = self.theta*180/np.pi,
            edgecolor='black',
            linewidth=1.0,
            animated=True,
            alpha=1,
            zorder=2)
        ax.add_patch(robot)
        name = 'data/snap%s.png'%str(self.time)
        ax.plot([self.goal[0]], [self.goal[1]], '*', color="red", markersize =15,linewidth=3.0)
        ax.set_aspect('equal')
        ax.set_xlim(-10.0, 60.0)
        ax.set_ylim(-10.0, 60.0)
        ax.set_xlabel(r'$x (m)$')
        ax.set_ylabel(r'$y (m)$')
        ax.grid(True)
        plt.savefig(name, dpi = 200)
        plt.cla()
        plt.close(figure)
        return figure
    


In [6]:
bot = agent(2,np.array([0,0]),np.array([50,50]),np.pi/4,10,7)
bot.opt_traj()

v_new:  9.899999861907286 w_new:  1.7235749819289082e-12
1 4859.578645687998
9.999999861907286 1.7235749819289082e-12
v_new:  1.3671178701104296e-07 w_new:  -1.2185796278997939e-10
2 4859.578643781945
0 0.7853981633974483 9.999999998619073 -1.2013438780805047e-10 [0.7071067810973961, 0.7071067810804065]
1 0.7853981633852625 9.999999998619073 -1.2013438780805047e-10 [1.4142135622034089, 1.4142135621521963]
2 0.7853981633730767 9.999999998619073 -1.2013438780805047e-10 [2.1213203433180383, 2.1213203432153693]
3 0.7853981633608909 9.999999998619073 -1.2013438780805047e-10 [2.8284271244412844, 2.828427124269926]
4 0.785398163348705 9.999999998619073 -1.2013438780805047e-10 [3.535533905573147, 3.535533905315866]
5 0.7853981633365192 9.999999998619073 -1.2013438780805047e-10 [4.242640686713626, 4.242640686353189]
6 0.7853981633243334 9.999999998619073 -1.2013438780805047e-10 [4.949747467862722, 4.949747467381895]
7 0.7853981633121476 9.999999998619073 -1.2013438780805047e-10 [5.6568542490204